# Markov chains for gradient estimation in the context of Restricted Boltzman Machine Learning

_Authors: Alan García and Sandra Burgos_

notebook explanation...

## Packages

## Datasets

For the development and testing of the RBM implementations the MNIST handwritten numbers dataset has been used...

## RBM Base

This is the main class of our implementation...

## RBM Contrastive Divergence

Here it is the implementation of the Contrastive Divergence algorithm for training Restricted Boltzman Machines...

## RBM Persistent Contrastive Divergence

Here it is the implementation of the Persisten Contrastive Divergence algorithm...